In [1]:
import base64
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pytesseract
import json

In [2]:
## OCR Code for Medical Prescription

In [3]:
#Load Image Button defination 

def load_image():
    global panel, img_label, text_box, filename
    filename = filedialog.askopenfilename(initialdir="/", title="Select file",
                                          filetypes=(("jpeg files", "*.jpg"), ("png files", "*.png"), ("bmp files", "*.bmp")))
    if filename:
        img = Image.open(filename)
        img = img.resize((250, 250), Image.LANCZOS)
        img = ImageTk.PhotoImage(img)

        if panel is None:
            panel = tk.Label(image=img)
            panel.image = img
            panel.grid(row=0, column=0)
        else:
            panel.configure(image=img)
            panel.image = img

        # Extract text using Tesseract
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        text = pytesseract.image_to_string(Image.open(filename))
        text_box.delete("1.0", "end")
        text_box.insert(tk.END, text)

In [4]:
#Convert to JSON Button

def convert_to_json():
    global text_box, filename
    if filename:
        desired_location = filedialog.asksaveasfilename(initialdir="/", title="Save JSON file",
                                             filetypes=(("JSON files", "*.json"), ("All files", "*.*")))
        if desired_location:
            text = text_box.get("1.0", "end-1c")
            with open(filename, 'rb') as f:
                image_data = f.read()
            image_data_base64 = base64.b64encode(image_data)  # Use base64 module
            json_data = {
                "filename": filename,
                "image_data": image_data_base64.decode('utf-8'),
                "extracted_text": text
            }
            with open(desired_location, 'w') as f:
                json.dump(json_data, f, indent=4)

In [5]:
# Cancel Button
def cancel():
    global panel, text_box, filename
    if panel is not None:
        panel.grid_forget()
        panel = None
    text_box.delete("1.0", "end")
    filename = None

In [6]:


root = tk.Tk()
root.title("Image to Text and JSON Converter")

panel = None
filename = None  # Initialize the filename variable

upload_button = tk.Button(root, text="Upload", command=load_image)
upload_button.grid(row=1, column=0)

convert_to_json_button = tk.Button(root, text="Convert to JSON", command=convert_to_json)
convert_to_json_button.grid(row=1, column=1)

cancel_button = tk.Button(root, text="Cancel", command=cancel)
cancel_button.grid(row=1, column=2)

text_box = tk.Text(root, height=10, width=40)
text_box.grid(row=0, column=1)

root.mainloop()
